In [1]:
import pandas as pd
from dataset_manager import DatasetManager

# 1 - Load de dados

In [2]:
import os

DATA_PATH = "./data"

def download_file(url, final_path=None):
    import urllib.request

    if not final_path:
        name = url.split("/")[-1]
        final_path = "{}/{}".format(DATA_PATH, name)
    urllib.request.urlretrieve(url, final_path)
    return final_path

def unzip_file(path_to_zip_file, final_path=None):
    import zipfile
    
    if not final_path:
        name = os.path.basename(path_to_zip_file).split(".")[0]
        final_path = "{}/{}".format(DATA_PATH, name)

    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(final_path)
    zip_ref.close()
    return final_path

def download_all(manager, force=False):
    for dataset in manager.list_datasets().to_dict('records'):
        if not os.path.exists(dataset["local_source"]) or force:
            if "zipped" in dataset["format"]:
                path_downloaded = download_file(dataset["source"])
                unzip_file(path_downloaded, dataset["local_source"])
            else:
                download_file(dataset["source"], dataset["local_source"])

def get_dataset_source(manager, identifier):
    dataset = manager.get_dataset(identifier)
    if os.path.isdir(dataset["local_source"]):
        return [os.path.join(dataset["local_source"], filename) for filename in os.listdir(dataset["local_source"])]
    elif os.path.isfile(dataset["local_source"]):
        return dataset["local_source"]
    else:
        raise Exception("No dataset exists as {}".format(dataset["local_source"]))
        

In [3]:
dataset_path = "./datasets"
manager = DatasetManager(dataset_path)
manager.list_datasets()

,description,format,identifier,local_source,source
0,Quantidade de projetos para a construção de cr...,json,aprovacao_construcao_creche,./data/aprovacao_construcao_creche.json,http://api.pgi.gov.br/api/1/serie/2555.json
1,Quantidade de creches e pré-escolas cujas obra...,json,construcao_creche,./data/construcao_creche.json,http://api.pgi.gov.br/api/1/serie/2620.json
2,Valor do investimento para a construção das cr...,json,investimento_em_construcao_creche,./data/investimento_em_construcao_creche.json,http://api.pgi.gov.br/api/1/serie/1703.json
3,Média de alunos por turmas em escolas e creche...,xls zipped,media_alunos_2010,./data/media_alunos_2010,http://download.inep.gov.br/informacoes_estati...
4,Média de alunos por turmas em escolas e creche...,xls zipped,media_alunos_2008,./data/media_alunos_2008,http://download.inep.gov.br/informacoes_estati...
5,Média de alunos por turmas em escolas e creche...,xls zipped,media_alunos_2009,./data/media_alunos_2009,http://download.inep.gov.br/informacoes_estati...


In [4]:
download_all(manager, False)

In [5]:
alunos = manager.get_dataset("media_alunos_2008")
alunos_local = get_dataset_source(manager, "media_alunos_2008")
alunos = pd.read_excel(alunos_local[0])

# 2 - Rename de colunas

In [6]:
alunos_renamed = alunos.rename(
    columns={
        "Unnamed: 0" : "ano",
        "Ministério da Educação":"regiao",
        "Unnamed: 2":"estado",
        "Unnamed: 3" : "cidade",
        "Unnamed: 4" : "cod_municipio",
        "Unnamed: 5" : "cod_escola",
        "Unnamed: 6" : "nome_da_escola",
        "Unnamed: 7" : "localizacao",
        "Unnamed: 8" : "rede",
        "Unnamed: 9" : "educacao_infantil - total",
        "Unnamed: 10" : "educacao_infantil - creche",
        "Unnamed: 11" : "educacao_infantil - pre-escola",
        "Unnamed: 12" : "ensino_fundamental - total",
        "Unnamed: 13" : "ensino_fundamental - anos_inic",
        "Unnamed: 14" : "ensino_fundamental - anos_finais",
        "Unnamed: 15" : "ensino_fundamental - 1ano",
        "Unnamed: 16" : "ensino_fundamental - 2ano",
        "Unnamed: 17" : "ensino_fundamental - 3ano",
        "Unnamed: 18" : "ensino_fundamental - 4ano",
        "Unnamed: 19" : "ensino_fundamental - 5ano",
        "Unnamed: 20" : "ensino_fundamental - 6ano",
        "Unnamed: 21" : "ensino_fundamental - 7ano",
        "Unnamed: 22" : "ensino_fundamental - 8ano",
        "Unnamed: 23" : "ensino_fundamental - 9ano",
        "Unnamed: 24" : "turmas_unificadas",
        "Unnamed: 25" : "ensino_medio - total_media",
        "Unnamed: 26" : "ensino_medio - 1ano",
        "Unnamed: 27" : "ensino_medio - 2ano",
        "Unnamed: 28" : "ensino_medio - 3ano",
        "Unnamed: 29" : "ensino_medio - 4ano",
        "Unnamed: 30" : "medio_nao_seriado"
    }
)

# 3 - Filtro

In [7]:
alunos_renamed_filtred = alunos_renamed[
    (~alunos_renamed["nome_da_escola"].isnull()) & (alunos_renamed["ano"] != "Ano")]